
<div align="center">

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dapivei/causal-infere/blob/main/sections/8_ML_and_Causality.ipynb)

</div>






# The Interplay between Machine Learning and Causal Inference

There is an increasing interest in  

1.   how machine learning techniques can improve causal effect estimation
2.   how causal inference principles can enhance machine learning algorithms, particularly for robustness

The aim of this lab is to give you a broad overview of existing work and research at the intersection of machine learning and causal inference.



## Machine Learning for Causality

The first step in performing causal inference is knowing the causal parameter we are trying to identify and estimate. In this class, we have been mostly focused on the average treatment effect (ATE):

\begin{equation}
ATE = \mathbb{E}[Y(1, U) - Y(0, U)]
\end{equation}

We can identify the above equation to form a statistical estimand by conditioning if $S \perp U$:

\begin{equation}
ATE = \mathbb{E}[Y(1, U)|S=1] - \mathbb{E}[Y(0, U)|S=0]
\end{equation}

Last week, we also learned that the identification of ATE can be facilitated through conditional independence.

\begin{equation}
S \perp U \ | \ C
\end{equation}

Remember, we cannot just make any (machine-learning) models as being "causal" models, but we know the conditions and assumptions under which causality is feasible: descriptive regression $→$ causal regression.

### Other Causal Parameters?

Actually, you have seen another causal parameter: individualized treatment effect (ITE). It is the most straightforward to write down, but it is also the most difficult one (almost impossible) to identify. Instead of estimating causal effects on a population level, the inidividual level effect is much more helpful as it underpins the promises of things like personalized medicine (one medication could be helpful on average but harmful for a particular individual).

\begin{equation}
ITE = Y_i(1, u) - Y_i(0, u)
\end{equation}

Instead, an easier quantity to learn is called conditional average treatment effect (CATE). For the rest of the lab, we will use a more standard machine learning notation involving $(Y, X, T)$ and not writing $U$ for simplicity.

\begin{equation}
CATE = \mathbb{E}[Y(1) - Y(0) | X]
\end{equation}

If we are simply leveraging the conditional independence assumption for identifying ATE, this can be written as the following:

\begin{equation}
ATE = \mathbb{E}_{X}\left[\mathbb{E}[Y(1) - Y(0) | X]\right]
\end{equation}

### Statistical Estimands

Given a dataset of $D = \{Y, X, T\}$, we can identify the above ATE and CATE with two major assumptions:


1. Ignorability: $Y(1), Y(0) \perp T | X$
2. Positivity: $p(T=t | X = \mathbf{x}) >0, ∀t, \mathbf{x}$

We have seen the first one! It reads as the treatment assignment does not influence the potential outcomes condition on the features $X$, i.e., the treatment group and the control group should have similar characteristics ($S \perp U | C$).

We haven't talked much about positivity yet as we are just beginning to touch on control variables, but this is intuitive: there always should be people taking all the treatment options among the subpopulation on which they are conditioned. For example, if all college graduates do not attend the job training program, this assumption will be violated.

We can derive the statistical estimand for CATE and ATE just as in lectures.

\begin{align}
CATE &= \mathbb{E}[Y(1) - Y(0) | X] \\
&= \mathbb{E}[Y(1)|X] - \mathbb{E}[Y(0) | X] \\
&= \mathbb{E}[Y(1)|T=1, X] - \mathbb{E}[Y(0) |T=0, X] \\
&= \mathbb{E}[Y|T=1, X] - \mathbb{E}[Y|T=0, X] \\
\end{align}

and,

\begin{align}
ATE &= \mathbb{E}_{X}\left[\mathbb{E}[Y(1) - Y(0) | X]\right] \\
&= \mathbb{E}_{X}\left[\mathbb{E}[Y|T=1, X] - \mathbb{E}[Y|T=0, X]\right] \\
\end{align}

### Advanced Estimation

When the assumptions hold, we can simply regress the outcome on the treatment and features, and we have shown in class that the coefficient for the treatment variable is the corresponding ATE. For CATE, we can add an interaction term between treatment and the conditioned feature like $\beta XT$.

However, what if the underlying true model is **not** a linear model?

In general, we can use any model class for estimating $\mathbb{E}[Y|T=1, X]$ and $\mathbb{E}[Y|T=0, X]$.

#### S-learner

We can use a single model $f(\mathbf{x}_i, t_i)$ to predict $Y$ based on $X$ and $T$.

![S-learner](https://github.com/dapivei/causal-infere/blob/main/images/S-learner.png?raw=true)

image credit: [link](https://csc2541-2022.github.io/lectures/CSC2541_lecture4_estimation.pdf)

Let's simulate some data!

In [1]:
import numpy as np
import pandas as pd

# simulate features
np.random.seed(42)
n = 5000
X = np.random.normal(0, 1, (n, 5))

# simulate treatment assignment (binary treatment)
T = np.random.binomial(1, 0.5, n)

# define true treatment effect function
def true_effect(x):
    return 2 * x[:, 0]

# simulate outcomes
Y = true_effect(X) * T + X[:, 1] + np.random.normal(0, 1, n)

# create a DataFrame
data = pd.DataFrame(X, columns=[f'X{i}' for i in range(1, 6)])
data['T'] = T
data['Y'] = Y

data.head()

,X1,X2,X3,X4,X5,T,Y
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,1,1.145004
1,-0.234137,1.579213,0.767435,-0.469474,0.542560,1,-0.081058
2,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,0,-1.284728
3,-0.562288,-1.012831,0.314247,-0.908024,-1.412304,1,-1.214069
4,1.465649,-0.225776,0.067528,-1.424748,-0.544383,0,1.651123


In [2]:
from sklearn.model_selection import train_test_split

# features and target
X_all = data.drop('Y', axis=1)
y_all = data['Y']

# perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# separate T and X in training and test sets
T_train = X_train['T']
X_train_features = X_train.drop('T', axis=1)

T_test = X_test['T']
X_test_features = X_test.drop('T', axis=1)

In [3]:
from sklearn.ensemble import RandomForestRegressor

# include T in the features
X_train_s = X_train.copy()
y_train_s = y_train.copy()

# train the S-Learner
model_s = RandomForestRegressor(random_state=42)
model_s.fit(X_train_s, y_train_s)

# prepare test data with T=1 and T=0
X_test_T1 = X_test.copy()
X_test_T1['T'] = 1

X_test_T0 = X_test.copy()
X_test_T0['T'] = 0

# predict outcomes
y_pred_T1 = model_s.predict(X_test_T1)
y_pred_T0 = model_s.predict(X_test_T0)

# estimate ITE
ite_s = y_pred_T1 - y_pred_T0

# compute ATE
ate_s = np.mean(ite_s)

print(f"Estimated ATE using S-Learner: {ate_s:.4f}")

Estimated ATE using S-Learner: -0.0192


What are some potential problems if we have high-dimensional $X$, i.e., $D$ is very large?

#### T-learner

Alternatively, we can build separate models $f_1(\mathbf{x}_i, T=1)$ and $f_0(\mathbf{x}_i, T=0)$ for the treatment and control group.

![T-learner](https://github.com/dapivei/causal-infere/blob/main/images/T-learner.png?raw=true)

image credit: [link](https://csc2541-2022.github.io/lectures/CSC2541_lecture4_estimation.pdf)

In [4]:
# split training data by treatment
train_data = X_train.copy()
train_data['Y'] = y_train

train_treated = train_data[train_data['T'] == 1]
train_control = train_data[train_data['T'] == 0]

# features and target for treated group
X_treated = train_treated.drop(['Y', 'T'], axis=1)
y_treated = train_treated['Y']

# features and target for control group
X_control = train_control.drop(['Y', 'T'], axis=1)
y_control = train_control['Y']

# train models
model_treated = RandomForestRegressor(random_state=42)
model_control = RandomForestRegressor(random_state=42)

model_treated.fit(X_treated, y_treated)
model_control.fit(X_control, y_control)

# predict outcomes on test set
y_pred_treated = model_treated.predict(X_test_features)
y_pred_control = model_control.predict(X_test_features)

# estimate ITE
ite_t = y_pred_treated - y_pred_control

# compute ATE
ate_t = np.mean(ite_t)

print(f"Estimated ATE using T-Learner: {ate_t:.4f}")

Estimated ATE using T-Learner: -0.0338


#### TAR-Net

What are some of the drawbacks of the above approaches?

![Tradeoff](https://github.com/dapivei/causal-infere/blob/main/images/Trade-off.png?raw=true)

[Johansson, Shalit, and Sontag, 2016](https://arxiv.org/abs/1606.03976) leveraged neural networks to overcome the above challenge via learning a common representation.

(Additional regularization needed to ensure balanced representations for both groups)

![TAR-Net](https://github.com/dapivei/causal-infere/blob/main/images/TAR-Net.png?raw=true)

image credit: [link](https://csc2541-2022.github.io/lectures/CSC2541_lecture4_estimation.pdf)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(42)

# prepare training data tensors
X_train_tensor = torch.tensor(X_train_features.values, dtype=torch.float32)
T_train_tensor = torch.tensor(T_train.values, dtype=torch.float32).unsqueeze(1)
Y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

# create DataLoader
train_dataset = TensorDataset(X_train_tensor, T_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# define TARNet model
class TARNet(nn.Module):
    def __init__(self, input_dim):
        super(TARNet, self).__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU()
        )
        self.outcome_0 = nn.Sequential(
            nn.Linear(100, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )
        self.outcome_1 = nn.Sequential(
            nn.Linear(100, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )

    def forward(self, x, t):
        phi = self.shared(x)
        y0 = self.outcome_0(phi)
        y1 = self.outcome_1(phi)
        y_pred = t * y1 + (1 - t) * y0
        return y_pred

    def predict(self, x):
        phi = self.shared(x)
        y0 = self.outcome_0(phi)
        y1 = self.outcome_1(phi)
        return y0, y1

# initialize model, loss function, and optimizer
model = TARNet(input_dim=5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# training loop
for epoch in range(100):
    model.train()
    total_loss = 0
    for X_batch, T_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch, T_batch)
        loss = criterion(y_pred, Y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

KeyboardInterrupt: 

In [7]:
# prepare test data tensors
X_test_tensor = torch.tensor(X_test_features.values, dtype=torch.float32)

# estimate ITE
model.eval()
with torch.no_grad():
    y0_pred, y1_pred = model.predict(X_test_tensor)
    ite_tar = (y1_pred - y0_pred).numpy().flatten()

# compute ATE
ate_tar = np.mean(ite_tar)

print(f"Estimated ATE using TARNet: {ate_tar:.4f}")

Estimated ATE using TARNet: -0.0584


In [8]:
# Compute true ATE
X_test_array = X_test_features.values
true_ite = true_effect(X_test_array)
true_ate = np.mean(true_ite)
print(f"True ATE:    {true_ate:.4f}")

True ATE:    -0.0276


### Double Machine Learning

TAR-Net still suffers a few problems:


*   does not support other machine learning models
*   only applies to binary treatment
*   no uncertainty quantification

Double ML provides a framework that leverage general machine learning models to obtain **unbiased** causal effect estimates with *fast convergence* and *confidence intervals*, and it is still among the best methods in causal effect estimation. (You just need to know such a method exists. Further reading: [link](https://econml.azurewebsites.net/spec/estimation/dml.html))

## Causal Inference for Machine Learning

We will not go into technical detail in this section but aim to provide you with a high-level intuition on how causality can help predictions.

The standard machine-learning pipeline assumes that the test examples are I.I.D as the training data distribution. However, in practice, this is often violated. For instance, we train an imaging model to read CT scans using data from NYU Langone, will this model do well if it is deployed in other countries w. different imaging equipment and lighting conditions?

![OOD-eg](https://github.com/dapivei/causal-infere/blob/main/images/OOD.png?raw=true)

image credit: [link](https://arxiv.org/abs/1911.08731)

We hope to build models that are robust to changes to test distribution, i.e., achieving out-of-distribution generalization. Why is this difficult?

During training, the model can easily pick up on predictive signals from features that are relevant in distribution, e.g., image background. This relationship will not hold in other data distributions.

However, causal relationships are reliable associations across different distributions. We can formalize it through causality so that we can build models that only rely on causal features/representations.